# Data Transforms

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import math
import torchvision

## Defining classes

In [5]:
class WineDataset(Dataset):
    def __init__(self, transform = None):
        # data loading
        xy= np.loadtxt('../datasets/wine.csv', delimiter = ',', dtype = np.float32, skiprows=1) #skiprows=1 for skipping the column names 
        self.X = xy[:,1:]
        self.y = xy[:,[0]] # [0] for keeping the shape as (n_samples,1)
        self.n_samples = self.X.shape[0]
        self.n_features = self.X.shape[1]

        self.transform = transform

    def __getitem__(self, index):
        sample = self.X[index],self.y[index]
        if self.transform:
            sample = self.transform(sample)
        return(sample)

    def __len__(self):
        return(self.n_samples)

In [6]:
class ToTensor:
    def __call__(self, sample):
        return(torch.from_numpy(sample[0]), torch.from_numpy(sample[1]))

In [9]:
class MulTransform:
    def __init__(self, factor):
        self.factor = factor
    def __call__(self, sample):
        inputs, targets = sample
        inputs *= self.factor
        return(inputs, targets)

### Using class `ToTensor` to convert to tensor

In [7]:
dataset = WineDataset(transform = ToTensor())
first_sample = dataset[0]
features, labels = first_sample
print(features, labels)

tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
        3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
        1.0650e+03]) tensor([1.])


In [13]:
dataset = WineDataset(transform = None)
first_sample = dataset[0]
features_old, labels_old = first_sample
print(features_old, labels_old)

[1.423e+01 1.710e+00 2.430e+00 1.560e+01 1.270e+02 2.800e+00 3.060e+00
 2.800e-01 2.290e+00 5.640e+00 1.040e+00 3.920e+00 1.065e+03] [1.]


In [11]:
composed = torchvision.transforms.Compose([ # Use list brackets
    ToTensor(), # Convert to tensor
    MulTransform(2) # Multiply inputs by 2
])

In [15]:
dataset = WineDataset(transform=composed)
first_sample = dataset[0]
features, labels = first_sample
print('Before (Without any transform):')
print(features_old, labels_old)
print('After (2 transforms in \'composed\'):')
print(features, labels)

Before (Without any transform):
[1.423e+01 1.710e+00 2.430e+00 1.560e+01 1.270e+02 2.800e+00 3.060e+00
 2.800e-01 2.290e+00 5.640e+00 1.040e+00 3.920e+00 1.065e+03] [1.]
After (2 transforms in 'composed'):
tensor([2.8460e+01, 3.4200e+00, 4.8600e+00, 3.1200e+01, 2.5400e+02, 5.6000e+00,
        6.1200e+00, 5.6000e-01, 4.5800e+00, 1.1280e+01, 2.0800e+00, 7.8400e+00,
        2.1300e+03]) tensor([1.])


Check out documentation on `torchvision.transforms` - https://pytorch.org/vision/0.9/transforms.html, for more inbuilt transforms available